# Code for R3C Exam

### Chad Weatherly
---

The purpose of this notebook is to have a testing ground for code locally before using it in the N3C Enclave

In [1]:
# These modules need to be installed. The requirements.txt file should take care of all that

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import torch
from torch import nn
from torch.utils.data import DataLoader
import session_info
from scipy.stats import sem
from functions import *

print(torch.__version__)

1.10.2


In [2]:
# Info for current Jupyter session modules
session_info.show(html=False)

-----
matplotlib          3.4.3
numpy               1.20.3
pandas              1.3.4
scipy               1.7.1
session_info        1.0.0
sklearn             1.0.1
torch               1.10.2
-----
IPython             7.27.0
jupyter_client      7.0.1
jupyter_core        4.8.1
jupyterlab          3.2.1
notebook            6.4.5
-----
Python 3.8.8 (default, Apr 13 2021, 12:59:45) [Clang 10.0.0 ]
macOS-10.16-x86_64-i386-64bit
-----
Session information updated at 2022-10-20 14:58


In [32]:
# Create data set

x = torch.rand(2000, 50, 1)
y = torch.rand(2000, 1)

x.shape

torch.Size([2000, 50, 1])

In [25]:
# Getting data ready, splitting and turning into tensors
rs = 22
kf = KFold(n_splits=5, shuffle=True, random_state=rs)
splits = kf.split(x, y)

k_fold = []
for s in splits:
    # print(len(s[0]), len(s[1]))
    
    x_train, x_test, y_train, y_test = x[s[0]], x[s[1]], y[s[0]], y[s[1]]

#     x_train.shape

    x_train_tn = []
    for i in range(len(x_train)):
        x_train_tn.append((x_train[i], y_train[i]))
    x_train_tn

    x_test_tn = []
    for i in range(len(x_test)):
        x_test_tn.append((x_test[i], y_test[i]))
    x_test_tn

    # Creating our DataLoaders, which are iterable objects
    batch_size = 35
    x_train = DataLoader(x_train_tn, batch_size=batch_size)
    x_test = DataLoader(x_test_tn, batch_size=batch_size)
    
    k_fold.append({'x_train':x_train, 'x_test':x_test})
    
len(k_fold)

5

In [26]:
k_fold[0]
# Each of the 5 K-folds are PyTorch dataloader objects, iterable objects

{'x_train': <torch.utils.data.dataloader.DataLoader at 0x7f967046c4f0>,
 'x_test': <torch.utils.data.dataloader.DataLoader at 0x7f9672e51730>}

In [27]:
for x_i, y_i in k_fold[0]['x_train']:
    print(x_i.shape)
    print(y_i.shape)
    break

torch.Size([35, 50])
torch.Size([35])


In [34]:
class lstm(nn.Module):
    """
    This class is the base for a binary classification LSTM model,
    which outputs a single value in the interval (0, 1). Assumes batch_first == TRUE
    for nn.LSTM
    INPUT:
        - input_size, the number of features each item in the input sequence has
                    (so if the input is a sequence of length L, how many features does
                    each item in L have?)
        - seq_length, a length of the sequence given
        - batch_size, the size of the batch used
        - hidden_size, the number of features in the hidden state of the LSTM
        - num_lstm_layers, the number of stacked LSTM layers to have
        - dropout, the proportion to use for dropout to prevent overfitting
        - num_lin_layers, the number of linear, fully-connected layers to have
                        following the LSTM layers
        - num_lin_units, the number of neurons/units to have in each of the above-mentioned
                        linear layers
        - activation, the type of activation function to use between linear layers. 
                    Must be one of these strings:
                    - 'elu' = Exponential Linear Unit
                    - 'relu' = Rectified Linear Unit
                    - 'sigmoid' = Sigmoid function
                    - 'tanh' = Hyperbolic Tangent function
    OUTPUT:
        - Prediction, a value in the interval (0, 1), corresponding to the 
                    two classes, coded as 0 and 1
    """
    def __init__(self, input_size, seq_length, batch_size,
                        hidden_size, num_lstm_layers, dropout,
                        num_lin_layers, num_lin_units, activation):
        # Super inherits all the attributes and methods
        # from the Neural Network __init__ method
        super(lstm, self).__init__()
        # So, now our Neural Network class inherits
        # all of the init attributes/methods from
        # the inherited class, nn.module
        
        # Essentially, our Neural Network class is the same as the nn.module
        # and then we are adding additional functionality to customize
        # it as something that we want
        
        # Submodules like layers or nn.Sequential are listed as attributes of
        # the overall nn.module class (in this case Neural Network)
        
        # Flatten takes each tensor and "flattens" it to create a 1-D vector
        # while ignoring batch, and it is used with nn.Sequential
        # So a tensor of size (32, 1, 25, 25) would be flattened to (32, 625)
        self.flatten = nn.Flatten()
        self.hidden_size = hidden_size
        self.num_lstm_layers = num_lstm_layers
        self.batch_size = batch_size
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        
        if activation == 'elu':
            self.act = nn.ELU()
        elif activation == 'relu':
            self.act = nn.ReLU()
        elif activation == 'sigmoid':
            self.act = nn.Sigmoid()
        elif activation == 'tanh':
            self.act = nn.Tanh()
        else:
            self.act = nn.Sigmoid()
        
        # Doing LSTM layer first
        self.rnn = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                           batch_first=True, dropout=dropout,
                           num_layers=num_lstm_layers)
        
        # Adding first layer to modules
        modules = []
        if num_lin_layers == 1:
            modules.append(nn.Linear(hidden_size*seq_length, 1))
            modules.append(nn.Sigmoid())
        elif num_lin_layers > 1:
            for layer in range(1, num_lin_layers+1):
                if layer == 1:
                    modules.append(nn.Linear(hidden_size*seq_length, num_lin_units))
                    modules.append(self.act)
                elif layer == num_lin_layers:
                    modules.append(nn.Linear(num_lin_units, 1))
                    modules.append(nn.Sigmoid())
                else:
                    modules.append(nn.Linear(num_lin_units, num_lin_units))
                    modules.append(self.act)

        # nn.Sequential is a container that holds the layers and chains them together for us
        self.linear_stack = nn.Sequential(*modules)
            # Modules held inside are submodules of nn.Sequential
            # nn.Sequential is treated as one submodule of Neural Network
    
    def forward(self, x, hidden=None):
        if hidden==None:
            self.hidden = (torch.zeros(self.num_lstm_layers,
                                        x.size()[0], 
                                        self.hidden_size).float().to(self.device),
                           torch.zeros(self.num_lstm_layers,
                                        x.size()[0], 
                                        self.hidden_size).float().to(self.device))
        else:
            self.hidden = hidden.to(self.device)
        
        out, self.hidden = self.rnn(x, self.hidden)
       
        out = self.flatten(out)
        logits = self.linear_stack(out)
        
        return logits, self.hidden
    
# Showing the structure of the Neural Network
temp_model = lstm(hidden_size=25).float().to(device)
print(temp_model)

TypeError: __init__() missing 8 required positional arguments: 'input_size', 'seq_length', 'batch_size', 'num_lstm_layers', 'dropout', 'num_lin_layers', 'num_lin_units', and 'activation'

In [ ]:
epochs = 1000
k_train_loss = []
k_test_loss = []
k_train_acc = []
k_test_acc = []

for k in range(len(k_fold)):
    print(f'K Fold: {k+1}')    
    
    model = lstm(hidden_size=25).float().to(device)
    loss_fn = nn.MSELoss()
    # loss_fn = nn.BCELoss(reduction='mean')
    # optimizer = torch.optim.SGD(model.parameters(), lr=0.0005, momentum=0.2)
    # optimizer = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.00005)
    
    x_train = k_fold[k]['x_train']
    x_test = k_fold[k]['x_test']
    
    train_loss = []
    train_acc = []
    test_loss = []
    test_acc = []
    for epoch in range(1, epochs+1):
        # Training
        model.train()
        all_loss = 0
        
        num_right = 0
        size = 0
        for i, (x_i, y_i) in enumerate(x_train):
            n, m = x_i.shape
            x_i = x_i.to(device).reshape(n, 1, m).type(torch.float)
            # print(x_i.shape)
            y_i = y_i.to(device).type(torch.float).reshape(y_i.shape[0], 1, 1)
            y_hat, _ = model(x_i, None)
            y_hat = y_hat[:,0].reshape(y_hat.shape[0], 1, 1)
            optimizer.zero_grad()
            # print(y_hat.shape, y_i.shape)
            loss = loss_fn(y_hat, y_i).float()
            loss = loss.type(torch.float)
            loss.backward()
            optimizer.step()
            
            # Getting training loss and accuracy
            all_loss += loss.item()
            
            # Getting Training Accuracy
            size += n
            y_hat_new = y_hat.clone().cpu().detach()
            num_right += (y_i.cpu() == y_hat_new.round()).sum().item()
        
        # If you want average loss, uncomment this line below
        # all_loss = all_loss / (len(x_train.dataset))
        
        # Average loss across each epoch
        train_loss.append(all_loss)
        train_acc.append(num_right / size)
        
        if epoch % np.round(epochs/5) == 0 or epoch==epochs:
            print(f'\tepoch:{epoch:4}')
            print(f'\t\tTraining loss: {all_loss:10.2f}')
            print(f'\t\tTraining Acc: {(num_right / size):10.2f}')
    
    
        # Test Loss
        model.eval()
        
        num_right = 0
        size = 0
        with torch.no_grad():
            all_loss = 0
            for i, (x_i, y_i) in enumerate(x_test):
                n, m = x_i.shape
                x_i = x_i.to(device).reshape(n, 1, m).type(torch.float)
                # print(x_i.shape)
                y_i = y_i.to(device).type(torch.float).reshape(y_i.shape[0], 1, 1)
                y_hat, _ = model(x_i, None)
                y_hat = y_hat[:,0].reshape(y_hat.shape[0], 1, 1)
                loss = loss_fn(y_hat, y_i).float()
                loss = loss.type(torch.float)
                

                # Getting loss and accuracy
                all_loss += loss.item()
                
                # Getting Training Accuracy
                size += n
                y_hat_new = y_hat.clone().cpu().detach()
                num_right += (y_i.cpu() == y_hat_new.round()).sum().item()
                
            # If you want average loss, uncomment this line below
            # all_loss = all_loss / (len(x_test.dataset))

            # Average loss and accuracy across each epoch
            test_loss.append(all_loss)
            test_acc.append(num_right / size)
            
        if epoch % np.round(epochs/5) == 0 or epoch==epochs:
            print(f'\t\tTesting loss: {all_loss:10.2f}')
            print(f'\t\tTesting Acc: {(num_right / size):10.2f}')
                
    k_train_loss.append(train_loss)
    k_test_loss.append(test_loss)
    k_train_acc.append(train_acc)
    k_test_acc.append(test_acc)
    

In [ ]:
colors = ['dimgray', 'lightcoral', 'khaki', 'royalblue', 'seagreen',
         'sandybrown', 'palegreen', 'darkorchid', 'maroon', 'aqua']

# Showing how the training/testing accuracy changes over epochs

plt.clf()
plt.figure(figsize=(10, 7.5))
plt.ylim([0, 1])
plt.xlabel('Epochs')
plt.ylabel('Training/Testing Accuracy')

plt.tight_layout()
plt.rcParams.update({'font.size': 15})

epochs = np.arange(len(k_train_acc[0]))
avg_train_acc = []
avg_test_acc = []

for a in range(len(k_train_acc[0])):
    
    train_total = 0
    test_total = 0
    w = len(k_train_acc)
    for k in range(w):  
        train_total += k_train_acc[k][a]
        test_total += k_test_acc[k][a]
        
    avg_train_acc.append(train_total / w)
    avg_test_acc.append(test_total / w)
    
plt.plot(epochs, avg_train_acc, color=colors[3], label='Training Accuracy', lw=3)
plt.plot(epochs, avg_test_acc, color=colors[8], label='Testing Accuracy', lw=3)

max_train = avg_train_acc.index(max(avg_train_acc))
max_test = avg_test_acc.index(max(avg_test_acc))

plt.scatter(epochs[max_train], avg_train_acc[max_train], color='maroon', zorder=2)
plt.text(epochs[max_train], avg_train_acc[max_train]+0.03,
         f'Max  = {round(avg_train_acc[max_train], 3)}',
        horizontalalignment='right')

plt.scatter(epochs[max_test], avg_test_acc[max_test], color='b', zorder=2)
plt.text(epochs[max_test], avg_test_acc[max_test]-0.06,
         f'Max  = {round(avg_test_acc[max_test], 3)}',
        horizontalalignment='center')
    
plt.title('Average Accuracy over Epochs')
plt.legend(loc=(1.04,0))

# plt.savefig('charts/teacher_training_loss.png', bbox_inches='tight')

plt.show()

In [ ]:
colors = ['dimgray', 'lightcoral', 'khaki', 'royalblue', 'seagreen',
         'sandybrown', 'palegreen', 'darkorchid', 'maroon', 'aqua']

# Showing how the training loss changes over epochs

plt.clf()
plt.figure(figsize=(10, 7.5))
plt.xlabel('Epochs')
plt.ylabel('Training Loss')

plt.tight_layout()
plt.rcParams.update({'font.size': 15})

epochs = np.arange(len(k_train_loss[0]))
for k in range(len(k_train_loss)):
    
    plt.plot(epochs, k_train_loss[k], color=colors[k], label='K' + str(k+1), lw=3)
    
plt.title('Training Loss over Epochs')
plt.legend(loc=(1.04,0))

# plt.savefig('charts/teacher_training_loss.png', bbox_inches='tight')

plt.show()

In [ ]:
colors = ['dimgray', 'lightcoral', 'khaki', 'royalblue', 'seagreen',
         'sandybrown', 'palegreen', 'darkorchid', 'maroon', 'aqua']

# Showing how the Testing loss changes over epochs

plt.clf()
plt.figure(figsize=(10, 7.5))
plt.xlabel('Epochs')
plt.ylabel('Testing Loss')

plt.tight_layout()
plt.rcParams.update({'font.size': 15})

epochs = np.arange(len(k_test_loss[0]))
for k in range(len(k_test_loss)):
    
    plt.plot(epochs, k_test_loss[k], color=colors[k], label='K' + str(k+1), lw=3)
    
plt.title('Testing Loss over Epochs')
plt.legend(loc=(1.04,0))

# plt.savefig('charts/teacher_testing_loss.png', bbox_inches='tight')

plt.show()

In [ ]:
train_loss = 0
test_loss = 0
all_train_loss = []
all_test_loss = []

best_loss = 10000

for k in range(len(k_fold)):
    print(f'K-Fold {k+1}')
    # Final Training Loss
    final_train_loss = round(k_train_loss[k][-1], 2)
    all_train_loss.append(final_train_loss)
    print(f'\tTraining Loss: {final_train_loss}')
    train_loss += final_train_loss
    # Final Testing Loss
    final_test_loss = round(k_test_loss[k][-1], 2)
    all_test_loss.append(final_test_loss)
    print(f'\t Testing Loss: {final_test_loss}')
    test_loss += final_test_loss
    
    avg_loss = (final_train_loss + final_test_loss) / 2 
    if avg_loss < best_loss:
        best_k = k
        best_loss = avg_loss
    
print()
print('----------------------------------')
print(f'Average Training Loss: {round(train_loss / len(k_train_loss), 2)}')
print(f' Average Testing Loss: {round(test_loss / len(k_train_loss), 2)}')
print()
print(f'Standard Error of Mean Training Loss: {sem(all_train_loss)}')
print(f'Standard Error of Mean Testing Loss: {sem(all_test_loss)}')